In [ ]:
import os
import shutil

main_directory = "/Users/rich/Desktop/AI-music/Datasets/"

#move files from subfolders to main folder
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name) 
            if file_name != ".DS_Store":
                destination_path = os.path.join(main_directory, file_name)
                shutil.move(file_path, destination_path)
#del subfolders
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)
    if os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
#rename files
count = 1
for file_name in os.listdir(main_directory):
    file_path = os.path.join(main_directory, file_name)
    if os.path.isfile(file_path):
        new_file_name = str(count) + os.path.splitext(file_name)[1]
        new_file_path = os.path.join(main_directory, new_file_name)
        os.rename(file_path, new_file_path)
        count += 1

In [1]:
import pretty_midi
import numpy as np
import glob
#讀取所有.mid檔案
midi_file_paths = glob.glob('/Users/rich/Desktop/AI-music/Datasets/*.mid')

#notes:音高
#velocities:力度
all_notes = []
all_velocities = []

#midi file processing 
#有些midi文件有问题，所以用try except
for file_path in midi_file_paths:
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        notes = []
        velocities = []
        #midi文件會有多種樂器
        #每個樂器都有多個音符
        #每個音符都有音高和力度
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes.append(note.pitch)
                velocities.append(note.velocity)

        all_notes.extend(notes)
        all_velocities.extend(velocities)
    except:
        pass

notes_array = np.array(all_notes)
velocities_array = np.array(all_velocities)

In [ ]:
#normalize
notes_normalized = notes_array.reshape(-1, 1) / float(notes_array.max())
velocities_normalized = velocities_array.reshape(-1, 1) / float(velocities_array.max())

# combine notes and velocities
combined_data = np.concatenate((notes_normalized, velocities_normalized), axis=1)

sequence_length = 10
inputs = []
targets = []

for i in range(len(combined_data) - sequence_length):
    inputs.append(combined_data[i:i+sequence_length])
    targets.append(combined_data[i+sequence_length])


#split data
split_index = int(0.8 * len(inputs)) 

train_inputs = inputs[:split_index]
train_targets = targets[:split_index]

test_inputs = inputs[split_index:]
test_targets = targets[split_index:]

In [ ]:
#build model
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(sequence_length, combined_data.shape[1])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(combined_data.shape[1], activation='sigmoid')
])

model.compile(optimizer='adam', loss='mse')

history = model.fit(train_inputs, train_targets, epochs=25, batch_size=64)
model.save('/Users/rich/Desktop/AI-music/Models/model02')

In [ ]:
#plot loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
#generate music
generated_sequence = []

start_index = np.random.randint(0, len(test_inputs)-1)
input_sequence = test_inputs[start_index]
sequence_length = 10

for _ in range(sequence_length):
    predicted = model.predict(np.expand_dims(input_sequence, axis=0))
    generated_sequence.append(predicted[0])
    input_sequence = np.concatenate((input_sequence[1:], np.expand_dims(predicted[0], axis=0)))

generated_sequence = notes_array.max() * generated_sequence


In [ ]:
#convert to midi
from music21 import stream, note, tempo

music_stream = stream.Stream()
#設置一個速度
default_tempo = tempo.MetronomeMark(number=120)

#提取音高還有持續時間
for sequence in generated_sequence:
    pitch = int(sequence[0])  
    duration = float(sequence[1])  

    n = note.Note()
    n.pitch.midi = pitch
    n.duration.quarterLength = duration

    music_stream.append(n)

music_stream.insert(0, default_tempo)

midi_filename = "generated_music.mid"
music_stream.write("midi", fp=midi_filename)
